## Configure

In [1]:
config = {
    # General
    'batchsize': 227,
    'shuffle': True,
    # Augmentations
    'flip': False,
    'ascale': True,
    'as_min': 0.6667,
    'as_max': 1.5,
    'rotate': False,
    'r_positions': 12,
    'translate': True,
    't_rate': 0.1,
    # Point clouds and kd-trees generation
    'steps': 10, # also control the depth of the network
    'lim': 1,
    'det': False,
    'gamma': 10.,
    # NN options
    'input_features': 'all', # 'all' for point coordinates, 'no' for feeding 1's as point features
    'n_f': [16, 
            32,  32,  64,  64,  128, 128, 256, 256, 512, 128], # representation sizes
    'n_output': 10,
    'l2_regularization': 1e-3,
    'learning_rate': 1e-3
}

## Load

In [2]:
import h5py as h5
import numpy as np

In [3]:
path2data = "/home/klokov/Data/ModelNet10_data/modelnet10.h5"
with h5.File(path2data, 'r') as hf:
    train_vertices = np.array(hf.get('train_vertices'))
    train_faces = np.array(hf.get('train_faces'))
    train_nFaces = np.array(hf.get('train_nFaces'))
    train_labels = np.array(hf.get('train_labels'))
    test_vertices = np.array(hf.get('test_vertices'))
    test_faces = np.array(hf.get('test_faces'))
    test_nFaces = np.array(hf.get('test_nFaces'))
    test_labels = np.array(hf.get('test_labels'))

## Build

In [4]:
from lib.nn.models import build_classification_network

train_fun, prob_fun, net, lr = build_classification_network(config)

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 970 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5110)


In [ ]:
from lib.nn.utilities import load_weights

load_weights('/home/klokov/Data/ModelNet10_data/models/test.pkl', net['output'])

## Train

In [ ]:
from sys import stdout
from time import time

from lib.iterators.iterator import Iterator
from lib.nn.utilities import acc_fun
from lib.nn.utilities import dump_weights

training

num_epochs = 25
num_batches = train_labels / config['batchsize']
num_save = 5

train_accuracy = np.zeros(num_epochs, dtype=np.float32)
test_accuracy = np.zeros(num_epochs/saven, dtype=np.float32)

for epoch in xrange(num_epochs):
    start_time = time()
    
    train_err = 0.
    train_acc = 0.
    for i in xrange(num_batches):
        batch = training_iterator.generate()
        train_err_batch, train_acc_batch = train_fun(batch[0], 
                                                     batch[1], batch[2], batch[3], batch[4], batch[5], 
                                                     batch[6], batch[7], batch[8], batch[9], batch[10],
                                                     batch[11])
        train_err += train_err_batch*len(batch[0])
        train_acc += train_acc_batch*len(batch[0])

        stdout.write('\rEpoch progress: {}/{}\tAccuracy: {:.2f} %\t\tLoss: {:.5f}'
                     .format(train_batchsize*(i+1),
                     len(train_nFaces)/train_batchsize*train_batchsize,
                     100*train_acc/(train_batchsize*(i+1)),
                     train_err/(train_batchsize*(i+1))))
        stdout.flush()
    stdout.write('\n')
    stdout.flush()
        
    train_accuracy[epoch] = 100*train_acc/np.float32(train_batchsize*i)
        
    print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time() - start_time))
    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err/(train_batchsize*i)))
    print("  train accuracy:\t\t{:.2f} %".format(train_accuracy[epoch]))
    if (epoch+1) % saven == 0:
        test_accuracy[epoch/saven] = acc_fun(test_vertices, test_faces, test_nFaces, test_labels, n_ens=10)
        print("  test accuracy:\t\t{:.2f} %".format(test_accuracy[epoch/saven]))

    if (epoch+1) % 1 == 0:
        dump_weights('RT_AS+TR_10.pkl')
        print '  Model saved!'